# TensorFlow Estimator amb Pandas

Creació d'un model d'aprenentatge automàtic utilitzant l'API Estimator de Tensorflow a partir de dades en format CSV utilitzant Pandas

### Preparació

In [ ]:
# Importació de paquets

import numpy as np
import pandas as pd
import tensorflow as tf

# Increment de la verbositat
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

print("Versió de Tensorflow:",tf.__version__)

In [ ]:
# Paràmetres

OUTDIR = './trained_model'  # Directori per als punts de control
MAX_STEPS = 2000            # Cicles d'entrenament

In [ ]:
# Lectura del dataset

df = pd.read_csv("https://storage.googleapis.com/ml_universities/california_housing_train.csv", sep=",")

In [ ]:
# Divisió del dataset

np.random.seed(seed=1)
msk = np.random.rand(len(df)) < 0.8
traindf = df[msk]
evaldf = df[~msk]

In [ ]:
# Funció per generar nous atributs

def add_more_features(df):
    df['rooms_per_house'] = df['total_rooms'] / df['households']
    return df

In [ ]:
# Funció d'entrada de Pandas

def make_input_fn(df, num_epochs):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x = add_more_features(df),     # Atributs
        y = df['median_house_value'],  # Etiqueta objectiu
        batch_size = 128,              # Nombre de registres amb què es calcula el gradient per cicle d'entrenament
        num_epochs = num_epochs,       # Nombre de vegades que s'utilitza el dataset durant l'entrenament
        shuffle = True,                # Redistribució dels registres
        queue_capacity = 1000,
        num_threads = 1
    )

In [ ]:
# Funció que defineix els atributs a TF

def create_feature_cols():
    return [
        tf.feature_column.numeric_column('housing_median_age'),
        tf.feature_column.bucketized_column(tf.feature_column.numeric_column('latitude'),
                                            boundaries = np.arange(32.0, 42, 1).tolist()),
        tf.feature_column.numeric_column('rooms_per_house'),
        tf.feature_column.numeric_column('median_income')
    ]

### Entrenament

In [ ]:
# Funció que crea l'estimador i defineix les tasques d'entrenament i avaluació

def train_and_evaluate(output_dir, num_train_steps):
    
    # Estimador
    estimator = tf.compat.v1.estimator.LinearRegressor(
        model_dir = output_dir,                   # Directori per als punts de control
        feature_columns = create_feature_cols()   # Atributs definits a TF
    )
    
    # Configuració del conjunt de dades d'entrenament
    train_spec = tf.estimator.TrainSpec(
        input_fn = make_input_fn(traindf, None),  # Funció d'entrada de dades
        max_steps = num_train_steps)              # Nombre màxim de cicles d'entrenament
    
    # Configuració del conjunt de dades d'avaluació
    eval_spec = tf.estimator.EvalSpec(
        input_fn = make_input_fn(evaldf, 1),      # Funció d'entrada de dades
        steps = None, 
        start_delay_secs = 1,                     # Retard en l'inici de l'avaluació
        throttle_secs = 5)                        # Període (s) entre avaluacions
    
    # Entrenament del model
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [ ]:
# Execució del model

tf.compat.v1.summary.FileWriterCache.clear() 
train_and_evaluate(OUTDIR, MAX_STEPS)

### Supervisió del model

In [ ]:
# Supervisió amb TensorBoard

%load_ext tensorboard
%tensorboard --logdir trained_model